This notebook contains the implementation of the models to:

- Make boundary boxes
- Crop the images
    - Store the cropped images in a folder, with the name of the shelf_image. All the cropped images should be there
- Try to eliminate the non-product images

Models:

- YOLOv11.n (10MB)
- YOLOv5 (148MB)

In [3]:
# Dependencies and Packages:

!pip install ultralytics
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/YOLOv5_SKU.pt" /content/
!git clone https://github.com/warun7/dotslash-repo.git
!cp "/content/dotslash-repo/YOLOv11_SKU.pt" /content/
!unzip /content/dotslash-repo/smart_cataloging.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'dotslash-repo' already exists and is not an empty directory.
Archive:  /content/dotslash-repo/smart_cataloging.zip
replace __MACOSX/._smart_cataloging? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
## Importing the models

import torch
import ultralytics
from ultralytics import YOLO



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
import os
import torch
from PIL import Image
import numpy as np
from pathlib import Path
import cv2

class ProductDetector:
    def __init__(self, yolov5_weights, yolov11n_weights, base_output_dir):
        """
        Initialize detector with both models

        Args:
            yolov5_weights (str): Path to YOLOv5 weights
            yolov11n_weights (str): Path to YOLOv11n weights
            base_output_dir (str): Base directory for all outputs
        """
        self.models = {
            'YOLOv5': self._load_model('yolov5', yolov5_weights),
            'YOLOv11': self._load_model('yolov11n', yolov11n_weights)
        }
        self.base_output_dir = base_output_dir

    def _load_model(self, model_type, weights_path):
        """Load YOLO model with specified weights"""
        if model_type == 'yolov5':
            model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
        else:  # yolov11n
            model = torch.hub.load('ultralytics/yolov11n', 'custom', path=weights_path)

        model.conf = 0.25  # confidence threshold
        model.iou = 0.45   # NMS IoU threshold
        return model

    def create_output_structure(self, image_name):
        """Create output directory structure for an image"""
        output_dirs = {}
        for model_name in self.models.keys():
            # Create model-specific directory
            model_dir = os.path.join(self.base_output_dir, model_name)
            os.makedirs(model_dir, exist_ok=True)

            # Create image-specific directory
            image_dir = os.path.join(model_dir, image_name)
            os.makedirs(image_dir, exist_ok=True)

            output_dirs[model_name] = image_dir

        return output_dirs

    def draw_boxes(self, image, detections):
        """Draw bounding boxes on image"""
        img = np.array(image)
        for det in detections.xyxy[0].cpu().numpy():
            x1, y1, x2, y2, conf, cls = map(int, det[:6])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f'{conf:.2f}', (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        return Image.fromarray(img)

    def crop_detections(self, image, detections, output_dir, image_name):
        """Crop and save detected products"""
        saved_paths = []
        for i, det in enumerate(detections.xyxy[0].cpu().numpy()):
            x1, y1, x2, y2, conf, cls = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Crop detection
            cropped = image.crop((x1, y1, x2, y2))

            # Save cropped image
            save_path = os.path.join(output_dir, f"{image_name}_P_{i+1}.jpg")
            cropped.save(save_path)
            saved_paths.append(save_path)

        return saved_paths

    def filter_non_products(self, image_path, min_size=50, aspect_ratio_range=(0.5, 2.0)):
        """Filter out likely non-product images"""
        img = Image.open(image_path)
        width, height = img.size

        if width < min_size or height < min_size:
            return False

        aspect_ratio = width / height
        if not (aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1]):
            return False

        return True

    def process_image(self, image_path):
        """Process single image with both models"""
        # Load image
        image = Image.open(image_path)
        image_name = Path(image_path).stem

        # Create output directories
        output_dirs = self.create_output_structure(image_name)

        # Process with each model
        for model_name, model in self.models.items():
            print(f"Processing {image_name} with {model_name}...")

            # Detect products
            detections = model(image)

            # Save image with bounding boxes
            bb_image = self.draw_boxes(image, detections)
            bb_path = os.path.join(output_dirs[model_name], f"{image_name}_BB.jpg")
            bb_image.save(bb_path)

            # Crop and save individual products
            cropped_paths = self.crop_detections(image, detections,
                                               output_dirs[model_name], image_name)

            # # Filter non-products
            # for path in cropped_paths:
            #     if not self.filter_non_products(path):
            #         os.remove(path)
            #         print(f"Removed likely non-product: {path}")

    def process_directory(self, input_dir):
        """Process all images in directory"""
        for image_file in os.listdir(input_dir):
            if image_file.lower().endswith(('.jpg', '.jpeg')):
                image_path = os.path.join(input_dir, image_file)
                self.process_image(image_path)

def main():
    # Configuration
    input_dir = "/content/smart_cataloging"
    base_output_dir = "detection_results"
    yolov5_weights = "/content/YOLOv5_SKU.pt"
    yolov11n_weights = "/content/YOLOv11_SKU.pt"

    # Initialize detector
    detector = ProductDetector(
        yolov5_weights=yolov5_weights,
        yolov11n_weights=yolov11n_weights,
        base_output_dir=base_output_dir
    )

    # Process all images
    detector.process_directory(input_dir)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-11-23 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5x summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


HTTPError: HTTP Error 404: Not Found

In [8]:
import os
import torch
from PIL import Image
import numpy as np
from pathlib import Path
import cv2

class ProductDetector:
    def __init__(self, yolov5_weights, base_output_dir):
        """
        Initialize detector with YOLOv5 model

        Args:
            yolov5_weights (str): Path to YOLOv5 weights
            base_output_dir (str): Base directory for all outputs
        """
        self.model = self._load_model(yolov5_weights)
        self.base_output_dir = base_output_dir

    def _load_model(self, weights_path):
        """Load YOLOv5 model with specified weights"""
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
        model.conf = 0.25  # confidence threshold
        model.iou = 0.45   # NMS IoU threshold
        return model

    def create_output_directory(self, image_name):
        """Create output directory structure for an image"""
        # Create image-specific directory
        image_dir = os.path.join(self.base_output_dir, image_name)
        os.makedirs(image_dir, exist_ok=True)
        return image_dir

    def draw_boxes(self, image, detections):
        """Draw bounding boxes on image"""
        img = np.array(image)
        for det in detections.xyxy[0].cpu().numpy():
            x1, y1, x2, y2, conf, cls = map(float, det)
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Add confidence score
            conf_text = f'{conf:.2f}'
            cv2.putText(img, conf_text, (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        return Image.fromarray(img)

    def crop_detections(self, image, detections, output_dir, image_name):
        """Crop and save detected products"""
        saved_paths = []
        for i, det in enumerate(detections.xyxy[0].cpu().numpy()):
            x1, y1, x2, y2, conf, cls = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Crop detection
            cropped = image.crop((x1, y1, x2, y2))

            # Save cropped image
            save_path = os.path.join(output_dir, f"{image_name}_P_{i+1}.jpg")
            cropped.save(save_path)
            saved_paths.append(save_path)

        return saved_paths

    def filter_non_products(self, image_path, min_size=50, aspect_ratio_range=(0.5, 2.0)):
        """Filter out likely non-product images"""
        img = Image.open(image_path)
        width, height = img.size

        if width < min_size or height < min_size:
            return False

        aspect_ratio = width / height
        if not (aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1]):
            return False

        return True

    def process_image(self, image_path):
        """Process single image with YOLOv5"""
        # Load image
        image = Image.open(image_path)
        image_name = Path(image_path).stem

        # Create output directory
        output_dir = self.create_output_directory(image_name)

        print(f"Processing {image_name}...")

        # Detect products
        detections = self.model(image)

        # Save image with bounding boxes
        bb_image = self.draw_boxes(image, detections)
        bb_path = os.path.join(output_dir, f"{image_name}_BB.jpg")
        bb_image.save(bb_path)

        # Crop and save individual products
        cropped_paths = self.crop_detections(image, detections, output_dir, image_name)

        # Filter non-products
        for path in cropped_paths:
            if not self.filter_non_products(path):
                os.remove(path)
                print(f"Removed likely non-product: {path}")

        print(f"Finished processing {image_name}")

    def process_directory(self, input_dir):
        """Process all images in directory"""
        for image_file in os.listdir(input_dir):
            if image_file.lower().endswith(('.jpg', '.jpeg')):
                image_path = os.path.join(input_dir, image_file)
                self.process_image(image_path)

def main():
    # Configuration
    input_dir = "/content/smart_cataloging"
    base_output_dir = "detection_results"
    yolov5_weights = "/content/YOLOv5_SKU.pt"

    # Initialize detector
    detector = ProductDetector(
        yolov5_weights=yolov5_weights,
        base_output_dir=base_output_dir
    )

    # Process all images
    detector.process_directory(input_dir)

if __name__ == "__main__":
    main()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-23 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5x summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processing store_44...
Removed likely non-product: detection_results/store_44/store_44_P_7.jpg
Removed likely non-product: detection_results/store_44/store_44_P_8.jpg
Removed likely non-product: detection_results/store_44/store_44_P_9.jpg
Removed likely non-product: detection_results/store_44/store_44_P_11.jpg
Removed likely non-product: detection_results/store_44/store_44_P_12.jpg
Removed likely non-product: detection_results/store_44/store_44_P_13.jpg
Removed likely non-product: detection_results/store_44/store_44_P_16.jpg
Removed likely non-product: detection_results/store_44/store_44_P_18.jpg
Removed likely non-product: detection_results/store_44/store_44_P_21.jpg
Removed likely non-product: detection_results/store_44/store_44_P_26.jpg
Removed likely non-product: detection_results/store_44/store_44_P_27.jpg
Removed likely non-product: detection_results/store_44/store_44_P_30.jpg
Removed likely non-product: detection_results/store_44/store_44_P_32.jpg
Removed likely non-product: det

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Removed likely non-product: detection_results/store_30/store_30_P_6.jpg
Removed likely non-product: detection_results/store_30/store_30_P_7.jpg
Removed likely non-product: detection_results/store_30/store_30_P_8.jpg
Removed likely non-product: detection_results/store_30/store_30_P_10.jpg
Removed likely non-product: detection_results/store_30/store_30_P_12.jpg
Removed likely non-product: detection_results/store_30/store_30_P_14.jpg
Removed likely non-product: detection_results/store_30/store_30_P_18.jpg
Removed likely non-product: detection_results/store_30/store_30_P_19.jpg
Removed likely non-product: detection_results/store_30/store_30_P_20.jpg
Removed likely non-product: detection_results/store_30/store_30_P_21.jpg
Removed likely non-product: detection_results/store_30/store_30_P_23.jpg
Removed likely non-product: detection_results/store_30/store_30_P_25.jpg
Removed likely non-product: detection_results/store_30/store_30_P_26.jpg
Removed likely non-product: detection_results/store_30

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


KeyboardInterrupt: 

In [12]:
import os
import torch
import numpy as np
from PIL import Image
from pathlib import Path
import cv2
from ultralytics import YOLO  # For YOLOv11

class BaseDetector:
    """Base class with common detection methods"""
    def __init__(self, base_output_dir, model_name):
        self.base_output_dir = os.path.join(base_output_dir, model_name)
        self.model_name = model_name
        os.makedirs(self.base_output_dir, exist_ok=True)

    def create_output_directory(self, image_name):
        """Create output directory structure for an image"""
        image_dir = os.path.join(self.base_output_dir, image_name)
        os.makedirs(image_dir, exist_ok=True)
        return image_dir

    def draw_boxes(self, image, detections):
        """Draw bounding boxes on image"""
        img = np.array(image)

        if isinstance(detections, torch.Tensor):
            # YOLOv5 style detections
            dets = detections.cpu().numpy()
        else:
            # YOLOv11 style detections
            dets = detections.boxes.data.cpu().numpy()

        for det in dets:
            x1, y1, x2, y2 = map(int, det[:4])
            conf = det[4]

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Add confidence score
            conf_text = f'{conf:.2f}'
            cv2.putText(img, conf_text, (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        return Image.fromarray(img)

    def crop_detections(self, image, detections, output_dir, image_name):
        """Crop and save detected products"""
        saved_paths = []

        if isinstance(detections, torch.Tensor):
            # YOLOv5 style detections
            dets = detections.cpu().numpy()
        else:
            # YOLOv11 style detections
            dets = detections.boxes.data.cpu().numpy()

        for i, det in enumerate(dets):
            x1, y1, x2, y2 = map(int, det[:4])

            # Crop detection
            cropped = image.crop((x1, y1, x2, y2))

            # Save cropped image
            save_path = os.path.join(output_dir, f"{image_name}_P_{i+1}.jpg")
            cropped.save(save_path)
            saved_paths.append(save_path)

        return saved_paths

    def filter_non_products(self, image_path, min_size=50, aspect_ratio_range=(0.5, 2.0)):
        """Filter out likely non-product images"""
        img = Image.open(image_path)
        width, height = img.size

        if width < min_size or height < min_size:
            return False

        aspect_ratio = width / height
        if not (aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1]):
            return False

        return True

class YOLOv5Detector(BaseDetector):
    def __init__(self, weights_path, base_output_dir):
        super().__init__(base_output_dir, "YOLOv5")
        self.model = self._load_model(weights_path)

    def _load_model(self, weights_path):
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
        model.conf = 0.25
        model.iou = 0.45
        return model

    def detect(self, image):
        return self.model(image).xyxy[0]

class YOLOv11Detector(BaseDetector):
    def __init__(self, weights_path, base_output_dir):
        super().__init__(base_output_dir, "YOLOv11")
        self.model = self._load_model(weights_path)

    def _load_model(self, weights_path):
        model = YOLO(weights_path)
        model.conf = 0.25
        model.iou = 0.45
        return model

    def detect(self, image):
        return self.model(image)[0]

class ProductDetectionPipeline:
    def __init__(self, yolov5_weights, yolov11_weights, base_output_dir):
        """Initialize both detectors"""
        self.detectors = {
            'YOLOv5': YOLOv5Detector(yolov5_weights, base_output_dir),
            'YOLOv11': YOLOv11Detector(yolov11_weights, base_output_dir)
        }

    def process_image(self, image_path):
        """Process single image with both models"""
        image = Image.open(image_path)
        image_name = Path(image_path).stem

        for detector_name, detector in self.detectors.items():
            print(f"Processing {image_name} with {detector_name}...")

            # Create output directory
            output_dir = detector.create_output_directory(image_name)

            # Detect products
            detections = detector.detect(image)

            # Save image with bounding boxes
            bb_image = detector.draw_boxes(image, detections)
            bb_path = os.path.join(output_dir, f"{image_name}_BB.jpg")
            bb_image.save(bb_path)

            # Crop and save individual products
            cropped_paths = detector.crop_detections(image, detections,
                                                   output_dir, image_name)

            # Filter non-products
            for path in cropped_paths:
                if not detector.filter_non_products(path):
                    os.remove(path)
                    print(f"Removed likely non-product: {path}")

            print(f"Finished processing {image_name} with {detector_name}")

    def process_directory(self, input_dir):
        """Process all images in directory with both models"""
        for image_file in os.listdir(input_dir):
            if image_file.lower().endswith(('.jpg', '.jpeg')):
                image_path = os.path.join(input_dir, image_file)
                self.process_image(image_path)

def main():
    # Configuration
    input_dir = "/content/smart_cataloging"
    base_output_dir = "detection_results"
    yolov5_weights = "/content/YOLOv5_SKU.pt"
    yolov11_weights = "/content/YOLOv11_SKU.pt"

    # Initialize pipeline
    pipeline = ProductDetectionPipeline(
        yolov5_weights=yolov5_weights,
        yolov11_weights=yolov11_weights,
        base_output_dir=base_output_dir
    )

    # Process all images
    pipeline.process_directory(input_dir)

if __name__ == "__main__":
    main()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-23 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5x summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


Processing store_44 with YOLOv5...


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_7.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_8.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_9.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_11.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_12.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_13.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_16.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_18.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_21.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_26.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_27.jpg
Removed likely non-product: detection_results/YOLOv5/store_44/store_44_P_30.jpg
Removed likely non-product: detection_resul

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_1.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_9.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_10.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_13.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_14.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_17.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_18.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_19.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_20.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_21.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_22.jpg
Removed likely non-product: detection_results/YOLOv5/store_42/store_42_P_23.jpg
Removed likely non-product: detection_resu

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_2.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_3.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_4.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_5.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_6.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_7.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_8.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_9.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_10.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_11.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_12.jpg
Removed likely non-product: detection_results/YOLOv5/store_23/store_23_P_13.jpg
Removed likely non-product: detection_results/YO

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_9.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_10.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_11.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_35.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_45.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_47.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_49.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_51.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_53.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_55.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_60.jpg
Removed likely non-product: detection_results/YOLOv5/store_36/store_36_P_61.jpg
Removed likely non-product: detection_res

In [10]:
!zip detection_results -r /content/detection_results

updating: content/detection_results/ (stored 0%)
  adding: content/detection_results/store_30/ (stored 0%)
  adding: content/detection_results/store_30/store_30_P_22.jpg (deflated 6%)
  adding: content/detection_results/store_30/store_30_P_62.jpg (deflated 3%)
  adding: content/detection_results/store_30/store_30_P_77.jpg (deflated 6%)
  adding: content/detection_results/store_30/store_30_P_135.jpg (deflated 3%)
  adding: content/detection_results/store_30/store_30_P_95.jpg (deflated 6%)
  adding: content/detection_results/store_30/store_30_P_61.jpg (deflated 5%)
  adding: content/detection_results/store_30/store_30_P_118.jpg (deflated 5%)
  adding: content/detection_results/store_30/store_30_P_81.jpg (deflated 5%)
  adding: content/detection_results/store_30/store_30_P_130.jpg (deflated 7%)
  adding: content/detection_results/store_30/store_30_P_70.jpg (deflated 5%)
  adding: content/detection_results/store_30/store_30_P_50.jpg (deflated 6%)
  adding: content/detection_results/store_3

In [ ]:
!cp /content/dotslash-repo /content/drive/MyDrive